In [83]:
import pandas as pd
from datetime import datetime, timedelta
import os
import openpyxl
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart, Reference
from openpyxl.worksheet.table import Table, TableStyleInfo

def generate_month_data(year):
    months = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
    all_month_data = []
    
    for month in range(1, 13):
        first_day = datetime(year, month, 1)
        last_day = (datetime(year, month + 1, 1) - timedelta(days=1)) if month < 12 else datetime(year + 1, 1, 1) - timedelta(days=1)
        
        dates = pd.date_range(start=first_day, end=last_day, freq='D')
        weekdays = [date for date in dates if date.weekday() <= 5]  # Include Monday to Saturday

        month_data = [{
            "Date": date.strftime('%d/%m/%Y'),
            "Day": date.strftime('%A'),
            "Credit": "",
            "Cash": "",
            "Total": ""
        } for date in weekdays]
        
        all_month_data.append((months[month-1], month_data))
    
    return all_month_data

# Generate data for 2025
data_2025 = generate_month_data(2025)
month_dfs = {month_name: pd.DataFrame(month_data) for month_name, month_data in data_2025}
total_workdays = sum(len(df) for df in month_dfs.values())

save_path = os.path.expanduser('~/Documents/untitled folder/Revenue Tracker.xlsx')

with pd.ExcelWriter(save_path, engine='openpyxl') as writer:
    workbook = writer.book
    full_year_data = []
    
    for month_name, df in month_dfs.items():
        worksheet = workbook.create_sheet(title=month_name)
        month_year = f'{month_name[:3]}-25'
        
        worksheet.append([month_year])
        worksheet.merge_cells(start_row=1, start_column=1, end_row=1, end_column=5)
        worksheet["A1"].fill = PatternFill(start_color='FCE4E4', end_color='FCE4E4', fill_type='solid')
        worksheet["A1"].font = Font(bold=True, size=14)
        worksheet["A1"].alignment = Alignment(horizontal="center", vertical="center")
        
        header_row = ["Date", "Day", "Credit", "Cash", "Total"]
        worksheet.append(header_row)
        for cell in worksheet[2]:
            cell.fill = PatternFill(start_color='B6D7A8', end_color='B6D7A8', fill_type='solid')
            cell.font = Font(bold=True, size=14)
            cell.alignment = Alignment(horizontal="center", vertical="center")

        for index, row in df.iterrows():
            worksheet.append([row['Date'], row['Day'], row['Credit'], row['Cash'], row['Total']])
            full_year_data.append(row)

        for row_idx in range(3, len(df) + 3):
            worksheet[f'E{row_idx}'] = f'=IF(AND(C{row_idx}="", D{row_idx}=""), "", C{row_idx}+D{row_idx})'
        
        totals_row = ["Totals", f"{len(df)} workdays", f'=SUM(C3:C{len(df)+2})',
                      f'=SUM(D3:D{len(df)+2})',
                      f'=SUM(E3:E{len(df)+2})']
        worksheet.append(totals_row)

        # Apply bold and font size 14 for the totals row
        for cell in worksheet[len(df)+3]:
            cell.font = Font(bold=True, size=14)
            cell.alignment = Alignment(horizontal="center", vertical="center")
            cell.fill = PatternFill(start_color='F4CCCC', end_color='F4CCCC', fill_type='solid')

        # Format the rows and remove the totals from the chart range
        for row_idx in range(3, len(df) + 3):
            for col_idx in range(2, 6):
                worksheet.cell(row=row_idx, column=col_idx).border = Border(left=Side(style='thin'), right=Side(style='thin'), top=Side(style='thin'), bottom=Side(style='thin'))
                
            for col_idx in range(1, 6):
                worksheet.cell(row=row_idx, column=col_idx).fill = PatternFill(start_color='D9EAD3', end_color='D9EAD3', fill_type='solid')

        for col in ['A', 'B', 'C', 'D', 'E']:
            worksheet.column_dimensions[col].width = 16  # Set column width to 16

        # Currency formatting and center alignment
        for row in worksheet.iter_rows(min_row=3, max_row=len(df) + 3, min_col=3, max_col=5):
            for cell in row:
                if isinstance(cell.value, (int, float)):
                    cell.number_format = '₪#,##0.00'
                    cell.alignment = Alignment(horizontal="center", vertical="center")
        
        worksheet.sheet_view.showGridLines = False

        # Add a stacked bar chart for the month (excluding totals)
        chart = BarChart()
        chart.type = "col"
        chart.grouping = "stacked"
        chart.overlap = 100
        chart.title = f"{month_name} Breakdown"
        chart.legend = None

        data = Reference(worksheet, min_col=3, min_row=2, max_col=4, max_row=len(df) + 2)  # Exclude the totals row
        categories = Reference(worksheet, min_col=1, min_row=3, max_row=len(df) + 2)  # Exclude totals row
        chart.add_data(data, titles_from_data=True)
        chart.set_categories(categories)

        worksheet.add_chart(chart, "G5")

    # **Full Year Data Sheet (Dynamic)**
    full_year_ws = workbook.create_sheet(title="Full Year Data")
    
    # Add headers and apply bold formatting to the Full Year Data Sheet
    header_row = ["Date", "Day", "Credit", "Cash", "Total"]
    full_year_ws.append(header_row)
    for cell in full_year_ws[1]:
        cell.font = Font(bold=True, size=14)
        cell.alignment = Alignment(horizontal="center", vertical="center")
        cell.fill = PatternFill(start_color='B6D7A8', end_color='B6D7A8', fill_type='solid')

   

    # Use cell references to pull data from each month's sheet
    for month_idx, (month_name, df) in enumerate(month_dfs.items(), start=2):
        for row_idx in range(3, len(df) + 3):  # Start from row 3 to skip the header rows and include the last row
            # Pull only the relevant data: Date, Day, Credit, Cash, and Total
            date_cell = f'{month_name}!A{row_idx}'
            day_cell = f'{month_name}!B{row_idx}'
            credit_cell = f'{month_name}!C{row_idx}'
            cash_cell = f'{month_name}!D{row_idx}'
            total_cell = f'{month_name}!E{row_idx}'
            full_year_ws.append([f'={date_cell}', f'={day_cell}', f'={credit_cell}', f'={cash_cell}', f'={total_cell}'])

    # Apply table formatting
    table = Table(displayName="FullYearTable", ref=f"A1:E{len(full_year_data)+2}")
    full_year_ws.add_table(table)

    # Format columns for the Full Year Data sheet
    for col in ['A', 'B', 'C', 'D', 'E']:
        full_year_ws.column_dimensions[col].width = 16  # Set column width to 16

    # Currency formatting and center alignment for Full Year Data sheet
    for row in full_year_ws.iter_rows(min_row=2, max_row=len(full_year_data)+2, min_col=3, max_col=5):
        for cell in row:
            if isinstance(cell.value, (int, float)):
                cell.number_format = '₪#,##0.00'
                cell.alignment = Alignment(horizontal="center", vertical="center")

    full_year_ws.sheet_view.showGridLines = False

    # **Summary Sheet**
    summary_data = []
    for month_name in month_dfs.keys():
        summary_data.append([month_name, f'={month_name}!C{len(month_dfs[month_name])+3}',
                             f'={month_name}!D{len(month_dfs[month_name])+3}',
                             f'={month_name}!E{len(month_dfs[month_name])+3}'])
    
    df_summary = pd.DataFrame(summary_data, columns=["Month", "Credit", "Cash", "Total"])
    df_summary.to_excel(writer, sheet_name="Summary", index=False)

    # Format the summary sheet
    summary_ws = writer.sheets["Summary"]
    for row_idx, row in enumerate(summary_ws.iter_rows(min_row=1, max_row=len(df_summary)+1, min_col=1, max_col=4), start=1):
        for cell in row:
            cell.alignment = Alignment(horizontal="center", vertical="center")
            cell.font = Font(size=14, bold=True)
            cell.fill = PatternFill(start_color='D9EAD3', end_color='D9EAD3', fill_type='solid')

    # Set column widths for the summary sheet
    for col in ['A', 'B', 'C', 'D']:
        summary_ws.column_dimensions[col].width = 16  # Set column width to 16

    # Add a stacked bar chart to the Summary sheet (exclude Total)
    chart = BarChart()
    chart.type = "col"
    chart.grouping = "stacked"
    chart.overlap = 100
    chart.title = "Monthly Breakdown"
    chart.legend = None

    data = Reference(summary_ws, min_col=2, min_row=1, max_col=3, max_row=len(df_summary) + 1)  # Exclude Total column
    categories = Reference(summary_ws, min_col=1, min_row=2, max_row=len(df_summary) + 1)
    chart.add_data(data, titles_from_data=True)
    chart.set_categories(categories)

    summary_ws.add_chart(chart, "F5")

    # Add totals row for the entire year in the Summary sheet
    total_row = ["Year Total", f'=SUM(B2:B{len(df_summary)+1})', f'=SUM(C2:C{len(df_summary)+1})', f'=SUM(D2:D{len(df_summary)+1})']
    summary_ws.append(total_row)

    # Format the final totals row in the Summary sheet
    for cell in summary_ws[len(df_summary) + 2]:
        cell.font = Font(bold=True, size=14)
        cell.alignment = Alignment(horizontal="center", vertical="center")
        cell.fill = PatternFill(start_color='F4CCCC', end_color='F4CCCC', fill_type='solid')
 
print(f"Excel file 'Revenue Tracker.xlsx' has been generated successfully and saved to: {save_path}")


Excel file 'Revenue Tracker.xlsx' has been generated successfully and saved to: /Users/mohab/Documents/untitled folder/Revenue Tracker.xlsx
